# COIN Project

Authors:

    - Mona
    - Roman
    - Nick
    - Mateo

# 1. Setup

### Packages

In [370]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import tweepy
#import dotenv
import os
import yaml
import datetime
from pprint import pprint
import time
import re


### Environment Variables

In [371]:
#dotenv.load_dotenv()

In [372]:
with open("../twitter.yml", "r") as f:
    config = yaml.safe_load(f)

In [373]:
config

{'accounts': ['Fridays4future',
  'FridayForFuture',
  'GretaThunberg',
  'Luisamneubauer',
  'AnnaUKSCN',
  'AnunaDe',
  'adelaidecharli2',
  'HollyWildChild',
  'AlexandriaV2005',
  'fff_europe',
  'FFFUnitedStates',
  'fff_digital'],
 'hashtags': {'green': ['EndCoal',
   'EndFossilFuels',
   'PeopleNotProfit',
   'NoMoreEmptyPromises',
   'UprootTheSystem',
   'FridaysForFuture',
   'ClimateAction',
   'ClimateJustice',
   'ClimateEmergency',
   'ClimateStrike',
   'SaveThePlanet'],
  'brown': ['climatescam',
   'climatechangehoax',
   'fakeclimate',
   'climatehoax',
   'globalwarmingisahoax'],
  'neutral': ['ClimateCrisis', 'ClimateChange', 'Climate', 'GlobalWarming']}}

### Database Connection

In [374]:
#TBD

### Twitter API Connection

In [375]:
#Research Access:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAGjtbgEAAAAAyx9oZIai1hXZ9OyDhUUFMZQivpc%3DUunewUXR9hw3nyKQhjqdmfg7zSAoa1nPv6WKLLSPB7OwKwYBP3'

#Initializing the client to request the Twitter API
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

#expansions = ['attachments.poll_ids', 'attachments.media_keys', 'author_id', 'entities.mentions.username', 'geo.place_id', 'in_reply_to_user_id', 'referenced_tweets.id', 'referenced_tweets.id.author_id']
expansions = ['attachments.media_keys','author_id','geo.place_id','referenced_tweets.id','referenced_tweets.id.author_id']
max_results = 500
media_fields = ["duration_ms", "height", "media_key", "preview_image_url", "type", "url", "width", "public_metrics", "non_public_metrics", "organic_metrics", "promoted_metrics", "alt_text"]
#next_token = ''#(str | None) – This parameter is used to get the next ‘page’ of results. The value used with the parameter is pulled directly from the response provided by the API, and should not be modified. You can learn more by visiting our page on pagination.
place_fields = ['country', 'country_code', 'full_name', 'geo', 'id', 'name', 'place_type']
#poll_fields #(list[str] | str | None) – poll_fields
#since_id #(int | str | None) – Returns results with a Tweet ID greater than (for example, more recent than) the specified ID. The ID specified is exclusive and responses will not include it. If included with the same request as a start_time parameter, only since_id will be used.
sort_order = 'recency'
start_time = '2017-01-01T00:00:00Z' #(datetime.datetime | str | None) – YYYY-MM-DDTHH:mm:ssZ (ISO 8601/RFC 3339). The oldest UTC timestamp from which the Tweets will be provided. Timestamp is in second granularity and is inclusive (for example, 12:00:01 includes the first second of the minute). By default, a request will return Tweets from up to 30 days ago if you do not include this parameter.
#end_time = '2019-07-15T23:00:00Z'
#tweet_fields = ["attachments", "author_id", "context_annotations", "conversation_id", "created_at", "entities", "geo", "id", "in_reply_to_user_id", "lang", "non_public_metrics", "public_metrics", "organic_metrics", "promoted_metrics", "possibly_sensitive", "referenced_tweets", "reply_settings", "source", "text", "withheld"]
tweet_fields = ["attachments","author_id", "conversation_id","created_at","referenced_tweets","geo","public_metrics"]
#until_id #(int | str | None) – Returns results with a Tweet ID less than (that is, older than) the specified ID. Used with since_id. The ID specified is exclusive and responses will not include it.
#user_fields can be extracted from the .includes attribute of the tweepy response
user_fields = ["created_at", "description", "location", "name", "pinned_tweet_id", "profile_image_url", "protected", "public_metrics", "url", "username", "verified", "withheld"]


# 2. Load and Transform Data

### 2.1 Load

In [376]:
#specify tweets to look up
'''query_list = ['EndCoal', 'EndFossilFuels', 'PeopleNotProfit', 'NoMoreEmptyPromises', 'UprootTheSystem', 
              'FridaysForFuture', 'ClimateAction', 'ClimateJustice', 'ClimateEmergency', 'ClimateStrike', 
              'SaveThePlanet', 'climatescam', 'climatechangehoax', 'fakeclimate', 'climatehoax', 
              'globalwarmingisahoax', 'ClimateCrisis', 'ClimateChange', 'Climate', 'GlobalWarming']'''

query_list = ['FridaysForFuture', 'ClimateAction', 'ClimateJustice', 'ClimateEmergency', 'ClimateStrike']


In [377]:
def search_all_tweets(query):
    response = client.search_all_tweets(
                               query=query
                              ,expansions=expansions
                              ,max_results=max_results
                              ,media_fields=media_fields
                              ,place_fields=place_fields
                              ,sort_order=sort_order
                              ,start_time=start_time
                              #,end_time=end_time
                              ,tweet_fields=tweet_fields
                              ,user_fields=user_fields
                            )
    return response

In [378]:
def search_all_tweets_with_next_token(query, next_token):
    response = client.search_all_tweets(
                               query=query
                              ,expansions=expansions
                              ,max_results=max_results
                              ,media_fields=media_fields
                              ,place_fields=place_fields
                              ,sort_order=sort_order
                              ,start_time=start_time
                              #,end_time=end_time
                              ,tweet_fields=tweet_fields
                              ,user_fields=user_fields
                              ,next_token=next_token
                            )
    return response

In [379]:
def fetch_all_data(query, iteration_count, next_token):

    """ Fetches iteration_count * 100 Tweets from Twitter API using the set query params, including the initial next_token
    
    Parameters:
    ------------
    * iteration_count     : Integer indicating the max number of requests 
    * next_token          : String containing the next token for pagination
    
    """
    
    # Initializing the return parameters
    tweets = []
    users = []
    ref_tweets = []
    media = []
    places = []
    
    for i in range(0, iteration_count):

        if(len(next_token) == 0):
            response = search_all_tweets(query)
        else:
            response = search_all_tweets_with_next_token(query, next_token)

        tweets.append(response.data)
        users.append(response.includes['users'])
        ref_tweets.append(response.includes['tweets'])
        if "media" in response.includes.keys():
            media.append(response.includes['media'])
        
        if('places' in response.includes.keys()):
            places.append(response.includes['places'])
            
        try:
            next_token = response.meta['next_token']
        except:
            print('No more Tweets to fetch')
            break


    # Logging the last next_token for pagination into console --> needed for later requesting of the next tweets
    if('next_token' in response.meta):
        print(response.meta['next_token'])
        
    return tweets, users, ref_tweets, media, places

In [380]:
def unpack_dict(unpack_dict):
    unpacked = [v for k, v in unpack_dict.items()]
    return unpacked
def unpack_attachment(attachment_dict):
    return "|".join(attachment_dict["media_keys"]) if "media_keys" in attachment_dict.keys() else np.nan
def unpack_geo(geo_dict):
    return geo_dict["type"], "|".join([str(x) for x in geo_dict["bbox"]])
def unpack_geo_2(geo_dict):
    return geo_dict["place_id"]

In [381]:
def query_tweets(query):    
    tweets, users, ref_tweets, media, places = fetch_all_data(query, 300, '')

    user_df = pd.DataFrame(data=[user for sublist in users for user in sublist])
    tweet_df = pd.DataFrame(data=[tweet for sublist in tweets for tweet in sublist])
    ref_tweet_df = pd.DataFrame(data=[tweet for sublist in ref_tweets for tweet in sublist])
    media_df = pd.DataFrame(data=[item for sublist in media for item in sublist])
    place_df = pd.DataFrame(data=[place for sublist in places for place in sublist])
    
    user_df[["followers_count", "following_count", "tweet_count", "listed_count"]] = user_df.apply(lambda x: unpack_dict(x["public_metrics"]), axis=1, result_type="expand")
    del user_df["public_metrics"]

    tweet_df[["retweet_count", "reply_count", "like_count", "quote_count"]] = tweet_df.apply(lambda x: unpack_dict(x["public_metrics"]), axis=1, result_type="expand")
    tweet_df["media_keys"] = tweet_df.loc[tweet_df["attachments"].notna()].apply(lambda x: unpack_attachment(x["attachments"]), axis=1, result_type="expand")
    del tweet_df["public_metrics"]
    del tweet_df["attachments"]
    del tweet_df["referenced_tweets"]

    ref_tweet_df[['retweet_count', 'reply_count', 'like_count', 'quote_count']] = ref_tweet_df.apply(lambda x: unpack_dict(x["public_metrics"]), axis=1, result_type="expand")
    ref_tweet_df["media_keys"] = ref_tweet_df.loc[ref_tweet_df["attachments"].notna()].apply(lambda x: unpack_attachment(x["attachments"]), axis=1, result_type="expand")
    ref_tweet_df['place_id'] = ref_tweet_df.loc[ref_tweet_df["geo"].notna()].apply(lambda x: unpack_geo_2(x["geo"]), axis=1, result_type="expand")
    del ref_tweet_df["public_metrics"]
    del ref_tweet_df["attachments"]
    del ref_tweet_df["geo"]
    del ref_tweet_df["referenced_tweets"]

    if "geo" in place_df.columns:
        place_df[["type", "coordinates"]] = place_df.loc[place_df["geo"].notna()].apply(lambda x: unpack_geo(x["geo"]), axis=1, result_type="expand")
        del place_df["geo"]
        
    tweet_df.text = tweet_df.text.apply(lambda x: re.sub(r"[^A-Za-z0-9\w\s:@,]", "", x))
    return user_df, tweet_df, ref_tweet_df, place_df, media_df

In [382]:
all_user = all_tweets = all_ref_tweets = all_places = all_media = pd.DataFrame()
for i in query_list:
    print(i)
    user_df, tweet_df, ref_tweet_df, place_df, media_df = query_tweets(i)
    all_user = pd.concat([all_user, user_df])
    all_tweets = pd.concat([all_user, tweet_df])
    all_ref_tweets = pd.concat([all_user, ref_tweet_df])
    all_places = pd.concat([all_user, place_df])
    all_media = pd.concat([all_user, media_df])
    
    #time.sleep(60)
    

FridaysForFuture
b26v89c19zqg8o3fpyqo6y2lfez3zzfj2zvwqlp0f4519
ClimateAction


Rate limit exceeded. Sleeping for 169 seconds.


b26v89c19zqg8o3fpywln07eeiuh5g5ony38yzbqeeh6l
ClimateJustice


Rate limit exceeded. Sleeping for 119 seconds.


b26v89c19zqg8o3fpe45y984n3fd66eshj1iqq5819bp9
ClimateEmergency


Rate limit exceeded. Sleeping for 142 seconds.


b26v89c19zqg8o3fpytot5byqjn9v9dejjg68s0gq0vb1
ClimateStrike


Rate limit exceeded. Sleeping for 33 seconds.


b26v89c19zqg8o3fpe18ak60yd03dlyi3wbea5gs7ykql


In [383]:
all_tweets.dropna(subset=['text'], inplace=True)
print('Number of tweets fetched (rows, columns):', all_tweets.shape)

Number of tweets fetched (rows, columns): (148492, 25)


### 2.2 Transform Data

In [384]:
#drop emtpy tweet columns
all_tweets.drop(['description', 'location', 'name', 'pinned_tweet_id', 'profile_image_url', 'protected',
                 'url', 'username', 'verified', 'followers_count', 'following_count','tweet_count',
                'listed_count'], axis=1, inplace=True)
print(all_tweets.shape)
print(all_user.shape)

(148492, 12)
(664636, 16)


In [385]:
#adjust naming and datatypes
all_tweets['author_id'] = [int(i) for i in all_tweets['author_id']]
all_user.rename(columns={"id": "author_id"}, inplace=True)
all_user.rename(columns={"created_at": "user_created_at"}, inplace=True)

In [386]:
#drop duplicate users
all_user_unique = all_user.groupby('author_id').mean()
all_user_unique.shape

(202962, 7)

In [387]:
#put tweet and user info in one file
twitter_dataset = pd.merge(all_tweets,all_user_unique, on='author_id')
twitter_dataset.shape

(106852, 19)

In [388]:
#save to csv
twitter_dataset.to_csv("twitter_dataset.csv", index=False)

In [389]:
#save dataframe to csv
#all_user.to_csv("tweets_by_user.csv", index=False)
#all_tweets.to_csv("tweets_by_hashtags.csv", index=False)
#all_ref_tweets.to_csv("tweets_reftweets.csv", index=False, header=False)
#all_places.to_csv("tweets_places.csv", index=False)
#all_media.to_csv("tweets_media.csv", index=False)